In [1]:
import guidance
import openai
import pandas as pd
import numpy as np
import torch
from transformers import BertConfig, BertTokenizerFast, BertForSequenceClassification
from XAI_Utils import MOR_Predictor
from FewShotPipeline import FewShotPipeline

In [2]:
# Load the pre-trained model and tokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = "path/to/mor_umlsbert_baseline.pt"
tokenizer_path = "path/to/UmlsBERT/checkpoint/"
config = BertConfig.from_pretrained(tokenizer_path)
model = BertForSequenceClassification.from_pretrained(model_path, config=config)
model.config.output_hidden_states = True
model.eval()
tokenizer = BertTokenizerFast.from_pretrained(tokenizer_path)

Predictor = MOR_Predictor(model, tokenizer, 512, device)

train_admission = pd.read_csv('path/to/CORe_Dataset/MP_IN_adm_train.csv')
test_admission = pd.read_csv('path/to/CORe_Dataset/MP_IN_adm_test.csv')

In [3]:
openai.api_key = "API_KEY"

In [4]:
program_explain = guidance('''
{{~#system~}}
You are a helpful assistant.
{{~/system~}}

{{#each prompt_list~}}
{{#user~}}
EXAMPLE:
{{this.text}}
LABEL: {{this.label}}
{{~/user~}}
{{/each}}

{{#user~}}
Given the TEST EXAMPLE:
{{test_prompt}}
The true LABEL for this example is: {{target_label}}
{{#block hidden=True}}
Remember not to refer to whole sections, but individual details.\
{{~/block}}
Can you explain the top 3 reason that LABEL is {{target_label}} for this TEST EXAMPLE?\
{{~/user~}}

{{#assistant~}}
{{gen 'explanation' temperature=0}}
{{~/assistant~}}
                           
{{#user~}}
Then, rank the top 3 pieces of information in the TEST EXAMPLE that could most likely contribute to the LABEL {{#if target_label == 'mortality'}}survived{{else}}mortality{{/if}}?\
{{#block hidden=True}}
Provide a rationale for you believe each piece of information is influential in determining the label.\
{{~/block}}
{{~/user~}}

{{#assistant~}}
{{gen 'opposite_explanation' temperature=0}}
{{~/assistant~}}

{{#user~}}
Now, based on the TEST EXAMPLE, rank the top 3 features that, if altered,\
would most likely change the LABEL to {{#if target_label == 'mortality'}}survived{{else}}mortality{{/if}}?\
{{#block hidden=True}}
Provide a rationale for each feature selection, detailing what change will be made and why it would have a significant impact on the label if changed.
{{~/block}}
{{~/user~}}

{{#assistant~}}
{{gen 'change_explanation' temperature=0}}
{{~/assistant~}}
''')


In [5]:
train_embeddings = [Predictor.cls_embed(text) for text in train_admission.text]
train_embeddings = np.array(train_embeddings)

In [6]:
guidance.llm = guidance.llms.OpenAI("gpt-4")
guidance.llms.Transformers.cache.clear()

In [7]:
# Convert numerical labels to descriptive terms
def convert_label(num_label):
    return 'survived' if num_label == 0 else 'mortality'

In [8]:
idx = 10
instance = test_admission.iloc[idx, 1]
target_label = test_admission.iloc[idx, -1]
merged_indexes, prompt_list = Predictor.knn_prompt(train_embeddings, train_admission, instance, target_label, 'cosine')
# Convert numerical label to descriptive label
descriptive_label = convert_label(target_label)
prediction = program_explain(prompt_list=prompt_list, test_prompt=instance, target_label = descriptive_label)

Stop program system You are a helpful assistant. user EXAMPLE:
 CHIEF COMPLAINT: Dyspnea on exertion

PRESENT ILLNESS: 72 y/o female with extensive past medical history (see below) who has been c/o progressively worsening dyspnea on exertion over the past six months. Most recent cardiac cath and echo revealed severe MR along with moderate AI.

MEDICAL HISTORY: Mitral Regurgitation and Aortic Insufficiency Aortic Dissection s/p Aortic Root Replacement [**2153**] Coronary Artery Disease s/p Coronary Artery Bypass Graft x 2 [**2155**], s/p Stents to left main and POBA of OM1 [**2155**] Complete Heart Block s/p PPM [**2160**] Hypertension Hyperlipidemia s/p Appendectomy s/p Tonsillectomy

MEDICATION ON ADMISSION: Lisinopril 20mg qd, Lopressor 50mg [**Hospital1 **], Lasix 20mg qd, Crestor 10mg qd, Norvasc 5mg qd, Aspirin 81mg qd

ALLERGIES: Penicillins / Codeine / Sulfamethoxazole/Trimethoprim

PHYSICAL EXAM: VS: 62 130/69 5'3" 136# Gen: WDWN elderly female in NAD HEENT: EOMI, PERRL, NCAT, OP benign Neck: Supple, FROM, -JVD, -bruit Chest: CTAB -w/r/r, well-healed MSI Heart: RRR 3/6 SEM Abd: Soft, NT/ND, +BS Ext: Warm, well-perfused -Edema, well-healed right EVH incision Neuro: A&O x 3, MAE, non-focal

FAMILY HISTORY: Mother with hypertension. Father died from brain tumor/cancer.

SOCIAL HISTORY: Retired. Quit smoking [**2152**]. Denies ETOH. 
LABEL: mortality user EXAMPLE:
 CHIEF COMPLAINT: chest pain, +ETT

PRESENT ILLNESS: 46yoM w/1 year history of exertional chest pain, had +ETT referred for cardiac catheterization.

MEDICAL HISTORY: HTN ^chol GERD

MEDICATION ON ADMISSION: Omeprazole 30'

ALLERGIES: Patient recorded as having No Known Allergies to Drugs

PHYSICAL EXAM: Admission VS T HR 70 BP 127/81 RR 20 Ht 6'2" Wt Gen NAD Neuro A&Ox3 Neck supple, no bruits Pulm CTA bilat CV RRR no murmurs Abdm soft, NT/ND/NABS Ext no edema or varicosities. Pulses 2+ throughout Discharge VS T 98.7 HR 100ST BP 112/62 RR 18 O2sat 94%RA Wt 92.5kg Gen NAD Neuro nonfocal exam Pulm CTA bilat CV RRR S1-S2, no MRG. Sternum stable, incision CDI Abdm soft NT/NABS Ext warm, well perfused. 1+Pedal edema bilat. Rt SVH site w/steri CDI

FAMILY HISTORY: sister had PCI at age 50 years

SOCIAL HISTORY: Married, works in sports store. Remote tobacco-quit 5 years ago, Occaisional ETOH 
LABEL: survived user EXAMPLE:
 CHIEF COMPLAINT: DOE and angina

PRESENT ILLNESS: 64 yo male with increasing anginal symptoms, s/p PCI, presenting for cadriac cath. This revealed LM 60%, LAD 100%, CX 80%, patent OM1 stent, RCA diffuse disease. Referred for surgical evaluation.

MEDICAL HISTORY: CAD with OM1 stent [**2169**] IMI [**2161**] intermittent sleep apnea COPD venous insufficiency with BLE edema porphyria anemia thrombocytopenia A Fib Hep C left leg ellulitis gout hemochromatosis

MEDICATION ON ADMISSION: plavix 75 mg daily ASA 325 mg daily erythropoeitin 60,000 units q week prn allopurinol 300 mg daily spironolactone 20 mg daily lasix 20 mg daily

ALLERGIES: Metoprolol / Heparin Agents

PHYSICAL EXAM: 6'0" 107.5 kg HR 74 RR 18 right 134/90 left 139/71 WD/WNM stasis changes to lower extremities with pigment changes HEENT unremarkable neck wupple with full ROM CTAB [**Last Name (un) **] no m/r/g abd soft, NT, ND +BS extrems warm, well-perfused no varicosities neuro grossly intact 2+ bil. fem/DP/PT/radials

FAMILY HISTORY: father with first MI at 35, died at 75

SOCIAL HISTORY: DIrector of Public Works lives with wife no tobacco or ETOH 
LABEL: mortality user EXAMPLE:
 CHIEF COMPLAINT: exertional angina referred for cardiac cath which revealed left main and 3VD. Then referred for bypass surgery

PRESENT ILLNESS: 63yoW c/o increasing DOE over past several months, recent +ETT referred for cardiac cath. Cardiac cath done on day of admission revealed LM(60%)and #VD(50%LAD,80%LCx,80%RCA). Referred to cardiac surgery.

MEDICAL HISTORY: HTN, Hyperlipidemia, GERD, Tonsillectomy, Appendectomy, TAH, bladder suspension

MEDICATION ON ADMISSION: Prilosec 20 QD/prn Tenormin 50 QD Lisinopr

In [9]:
idx = 1686
instance = test_admission.iloc[idx, 1]
target_label = test_admission.iloc[idx, -1]
merged_indexes, prompt_list = Predictor.knn_prompt(train_embeddings, train_admission, instance, target_label, 'cosine')
# Convert numerical label to descriptive label
descriptive_label = convert_label(target_label)
prediction = program_explain(prompt_list=prompt_list, test_prompt=instance, target_label = descriptive_label)

Stop program system You are a helpful assistant. user EXAMPLE:
 CHIEF COMPLAINT: Headache

PRESENT ILLNESS: The patient is a 64 year-old male with a h/o HTN and hypercholesterolemia who stopped his medications secondary to financial difficulties who was transferred from [**Hospital3 **] for a ICH. Per report, he was at a wedding this evening when he suddenly became confused and combative complaining of severe headache. He presented to [**Hospital3 3583**] via EMS and continued to be combative and confused. He was moving all extremities but was complaining of severe headache located in the back of his head. He was intubated and a head CT there showed, per report, a large IVH with blood in the 3rd and 4th ventricles possibly coming from the right caudate head, temporal horns mildly enlarged with hydrocephalus and cerebral edema. He has elevated blood pressures with SBPs in the 250s that was non-responsive to labetalol, but responded to propofol. At this time, he was transferred to [**Hospital1 18**]. In the [**Hospital1 18**] ED, he was hypertensive on arrival with BP 180s, intubated and sedated on propofol.

MEDICAL HISTORY: HTN hypercholesterolemia

MEDICATION ON ADMISSION: None

ALLERGIES: Patient recorded as having No Known Allergies to Drugs

PHYSICAL EXAM: On admission: O: T: BP: 180/120 HR: 74 AC 1.0/14/500/5 Gen: Intubated and sedated on propofol HEENT: Pupils: Nonreactive but on propofol. Neck: Supple. Lungs: CTA bilaterally. Cardiac: RRR. S1/S2. Abd: Obese, soft, BS+ Extrem: Warm and well-perfused. Neuro: Unable to be assessed at this time.

FAMILY HISTORY: Per sister there is a significant family history of heart disease. Other family members with DM. Also FH of difficult to control HTN and ICH.

SOCIAL HISTORY: The patient lives alone and has no children. Per sister who was at the bedside the patient has been estranged from his family for ~8 years after a dispute over their house after their parent's death. The sister reports the patient having a history of isolating himself and holding grudges. He cleaned homes and worked for himself. He had a number of friends. [**Name (NI) **] has, per report, no smoking history, occasional etoh use and no known drug use. There is not an official HCP however there is next of [**Doctor First Name **] in the greater [**Location (un) 86**] area [**Name (NI) **] [**Name (NI) **], sister [**Telephone/Fax (1) 83725**] 
LABEL: survived user EXAMPLE:
 CHIEF COMPLAINT: transfer from OSH for large brain hemorrhage

PRESENT ILLNESS: 83 year old female with h/o mild Alzheimer's disease, L CEA in 05, HTN, elevated lipids, bladder CA (no known metastasis), who presents as a transfer from OSH with unresponsiveness, L blown pupil and a very large intra cranial hemorrhage on CT. Daughter reports that she was last seen well yesterday. This morning she spoke to her over the phone and she sounded weird. She replied "OK doc", she was speaking slowing and then stopped talking. She went to her house and found her on the floor in the kitchen, snoring. She was taken to OSH where she was found to have a dilated left pupil, she was intubated and a CT head showed a devastating hemorrhage affecting almost all right hemisphere with significant shift. She was transferred here for evaluation by neurosurgery who found it an extremely poor surgical candidate. The family opted to make her CMO.

MEDICAL HISTORY: mild Alzheimer's disease, L CEA in 05 HTN, elev lipids, bladder CA (no known metastasis),

MEDICATION ON ADMISSION: HCTZ 25mg QD Zetia 10mg QD Amlodipine 10mg QD Plavix 75mg QD Synthriod 100mg QD Atenolol 50mg QD

ALLERGIES: Statins-Hmg-Coa Reductase Inhibitors

PHYSICAL EXAM: :99.1 intial BP:240/92 -> 180/85 HR:59 On Mechanical Ventilation R:15 O2Sats 100% on vent Gen: Intubated elderly lady, non-responsive. Neck: In cervical collar Lungs: Mechanical breath sounds b/l, present on both sides Cardiac: RRR. S1/S2. Abd: Soft, NT, BS+ Extrem: Warm and well-perfused.

FAMILY HISTORY: NC

SOCIAL HISTORY: Patient r